# RMC for linear systems
RMC can deal with linear systems of the form
$$x =Ax +b$$
where the coefficients of $A$ are sufficiently small. This very similar but simpler then how Ulam and Neumann did it.

Lets solve 
$$x = 
\begin{pmatrix}
-\frac{1}{8} & \frac{1}{8} \\
\frac{1}{8} & \frac{1}{8}
\end{pmatrix}x
+
\begin{pmatrix}
1 \\
\frac{3}{4}
\end{pmatrix}
$$
with RMC. <br>
This has solution x = [1,1] <br>
The basic idea is to add a random row selector matrix with expectation $I$. (still have to write it out)

In [4]:
from random import random, randint
import numpy as np

In [40]:
def x1():
    r = randint(0,3)
    if r == 0:
        return -0.5*x1() +1
    elif r==1:
        return 0.5*x2() +1
    else:
        return 1
    
def x2():
    r = randint(0,3)
    if r == 0:
        return 0.5*x1() +3/4
    elif r==1:
        return 0.5*x2() +3/4
    else:
        return 3/4

In [73]:
res = 0
for _ in range(1000):
    res += x1()/1000
print(res)

res = 0
for _ in range(1000):
    res += x2()/1000
print(res)

1.0065107421874995
0.9821152343750039


Same problem but with memory (taking the average inside recursion is possible but harder)
You can reuse every recursion 

In [196]:
memo1 = []
memo2 = []

def x1memo():
    r = randint(0,3)
    if r == 0:
        res = -0.5*x1memo() +1
    elif r==1:
        res = 0.5*x2memo() +1
    else:
        res = 1
    memo1.append(res)
    return res
    
def x2memo():
    r = randint(0,3)
    if r == 0:
        res= 0.5*x1memo() +3/4
    elif r==1:
        res= 0.5*x2memo() +3/4
    else:
        res= 3/4
    memo2.append(res)
    return res

In [236]:
memo1 = []
memo2 = []
for _ in range(1000):
    x1memo()
    x2memo()

print(len(memo1))
print(sum(memo1)/len(memo1))
print(len(memo2))
print(sum(memo2)/len(memo2))


2019
1.002028096365775
1953
0.9923935131848438


Now add dynamic control variate so we get something similar to sequential Monte Carlo

In [465]:
memo1 = []
memo2 = []
def x1memo_control(x1control, x2control):
    r = randint(0,3)
    if r == 0:
        res = -0.5*(x1memo_control(x1control,x2control)-x1control) - x1control/8 + x2control/8+1
    elif r==1:
        res = 0.5*(x2memo_control(x1control,x2control)-x2control) - x1control/8 + x2control/8+1
    else:
        res = 1+x1control/8 - x2control/8
    memo1.append(res)
    return res
    
def x2memo_control(x1control,x2control):
    r = randint(0,3)
    if r == 0:
        res= 0.5*(x1memo_control(x1control,x2control)-x1control) +x1control/8 + x2control/8 +3/4
    elif r==1:
        res= 0.5*(x2memo_control(x1control,x2control)-x2control) +x1control/8 + x2control/8+3/4
    else:
        res= 3/4+x1control/8 + x2control/8
    memo2.append(res)
    return res

In [560]:
x1control = 0
x2control = 0
# you can have finer control over the outer and inner iteration
for _ in range(5):
    memo1 = []
    memo2 = []
    for _ in range(1):
        x1memo_control(x1control,x2control)
        x2memo_control(x1control,x2control)
    x1control = sum(memo1)/len(memo1)
    x2control = sum(memo2)/len(memo2)

print(x1control)
print(x2control)


1.0001935437321663
1.000024475157261
